In [1]:
from glob import glob
from collections import Counter
import json
import datetime
from html.parser import HTMLParser
import pandas as pd
pd.options.display.max_colwidth = 500

In [18]:
class MLStripper(HTMLParser):

	def __init__(self):
		self.reset()
		self.strict = False
		self.convert_charrefs = True
		self.fed = []

	def handle_data(self, d):
		self.fed.append(d)

	def get_data(self):
		return ''.join(self.fed)

def strip_tags(html):

	s = MLStripper()
	s.feed(html)
	return s.get_data()
    
def read_json_tweets_file(myjsontweetfile, reqlang=['en','tr'], linecount=-1):
	"Each tweet should be in a line. The parameter linecount restricts number of tweets in case it is not -1."

	ftwits = []
	lang_cntr = Counter()
	with open(myjsontweetfile) as jfile:
		for i, ln in enumerate(jfile):
			if i == linecount:  # restrict line numbers for test
				break
			t = json.loads(ln)

			if ("lang" in t):
				lang_cntr[t["lang"]] += 1
			else:
				lang_cntr["NoLang"] += 1

			if ("lang" in t) and (t["lang"] not in reqlang):
				continue
			elif ("lang" not in t) or (t["lang"] in reqlang):
				t["created_at"] = datetime.datetime.strptime(t["created_at"], "%a %b %d %H:%M:%S +0000 %Y")
				# if t["created_at"].strftime("%Y-%m-%d") in flood_AnomBreakoutDaysList:
				if "entities" in t and "media" in t["entities"]:
					for tm in t["entities"]["media"]:
						if tm["type"] == 'photo':
							t["entity_type"] = 'photo'
							break

				if "entities" in t and "hashtags" in t["entities"]:
					t["entity_hashtags"] = [ehs["text"] for ehs in t["entities"]["hashtags"]]
			
				if "entities" in t and "user_mentions" in t["entities"]:
					t["entity_mentions"] = [ems["screen_name"] for ems in t["entities"]["user_mentions"]]


				# if "entities" in t and "urls" in t["entities"]:
				# 	t["entity_urls"] = [ers["display_url"] for ers in t["entities"]["urls"]]

				if "entities" in t and "urls" in t["entities"]:
					t["entity_urls"] = [ers.get("display_url",None) for ers in t["entities"]["urls"]]
                    
				if 'full_text' in t:
					t['text'] = t['full_text']


				try:
					if "place" in t:
						t["country"] = t["place"]["country"]
				except:
					pass
				if "retweeted_status" in t:
					t["is_retweet"] = True
				else:
					t["is_retweet"] = False

				
				if 'source' in t:
					t["device"] = strip_tags(t["source"])

				t["user_id"] = t["user"]["id_str"]

				if "followers_count" in t["user"]:
					t["user_followers"] = t["user"]["followers_count"]

				if "friends_count" in t["user"]:
					t["user_following"] = t["user"]["friends_count"]
                    
				if 'location' in t['user']:
					t['user_location'] = t['user']['location']
                    
				if 'name' in t['user']:
					t['user_name'] = t['user']['name']
                    
				if 'description' in t['user']:
					t['user_description'] = t['user']['description']
                    
				if 'statuses_count' in t['user']:
					t['user_statuses_count'] = t['user']['statuses_count']
                    
				if 'created_at' in t['user']:
					t['user_created_at'] = t['user']['created_at']
                    
                    
				if 'extended_tweet' in t:
					t['text'] = t['extended_tweet']['full_text']
                    
				if 'place' in t:
					t['tweet_location'] = t['place']['full_name']

				t["screen_name"] = t["user"]["screen_name"]

				t2 = {k:str(v) for k,v in t.items() if k in ["entity_type","entity_hashtags","entity_mentions","entity_urls",\
							"country","created_at","text","in_reply_to_user_id","id_str","user_id",\
							"user_followers","user_following", "coordinates", "is_retweet","device",\
							"screen_name",'user_location','tweet_location','user_name',\
							"user_description","user_statuses_count","user_created_at"]}
                
				# print(i, end = ',')
				ftwits.append(t2)  # .splitlines()
		print("Number of documents per languge:", lang_cntr)
		return ftwits

In [19]:
available_output_files = list(glob("_tweets_.json.xlsx")) + glob("_tweets_.json.csv")
print(available_output_files)

[]


In [20]:
ftwits = []
for twfile in glob("all_tweets_2021*.json"):
#     if twfile not in ["202003030549_COVID-19_tweets_.json","202002291512_coronavirus_tweets_.json",
#                      "202003021010_outbreak_tweets_.json","202003022001_2019-nCoV_tweets_.json",]
    print(twfile)
    ftwits = read_json_tweets_file(twfile)
    print("Total tweets until now:", len(ftwits))
    
    tw_df = pd.DataFrame(ftwits)
    print(len(tw_df))
    tw_df.drop_duplicates(subset=['id_str'], inplace=True)
    tw_df.drop_duplicates(subset=['text'], inplace=True)
    tw_df['text'] = tw_df.text.str.replace(r'[\n\t\r]+'," ")
    print(len(tw_df))
    
    if twfile+".csv" not in available_output_files:
        tw_df.to_csv(twfile+".csv", sep='\t')
        
    if twfile+".xlsx" not in available_output_files:
        tw_df.to_excel(twfile+".xlsx")

all_tweets_20210810.json
Number of documents per languge: Counter({'tr': 24250})
Total tweets until now: 24250
24250


<ipython-input-20-53d32982fcc4>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  tw_df['text'] = tw_df.text.str.replace(r'[\n\t\r]+'," ")


23927
